In [195]:
import pandas as pd
import numpy as np

DATA_PATH = "LoLesports_data/"
SEED = 42

teams_train = pd.read_csv(f"{DATA_PATH}teams_train.csv")
teams_test = pd.read_csv(f"{DATA_PATH}teams_test.csv")

teams_train_target = pd.read_csv(f"{DATA_PATH}teams_train_target.csv")
teams_test_target = pd.read_csv(f"{DATA_PATH}teams_test_target.csv")

## 상대 팀 추가

In [196]:
temp_opp_teams = teams_train.groupby("gameid")["teamname"].transform(lambda x: x.iloc[::-1].values).to_frame("opp_teamname")
teams_train = pd.concat([teams_train, temp_opp_teams], axis=1)
temp_opp_teams = teams_test.groupby("gameid")["teamname"].transform(lambda x: x.iloc[::-1].values).to_frame("opp_teamname")
teams_test = pd.concat([teams_test, temp_opp_teams], axis=1)

## 날짜 추가

In [197]:
teams_train["date"] = pd.to_datetime(teams_train["date"])
teams_test["date"] = pd.to_datetime(teams_test["date"])

teams_train["year"] = teams_train["date"].dt.year
teams_train["month"] = teams_train["date"].dt.month
teams_train["day"] = teams_train["date"].dt.day

teams_test["year"] = teams_test["date"].dt.year
teams_test["month"] = teams_test["date"].dt.month
teams_test["day"] = teams_test["date"].dt.day

## 데이터 타입 변경

In [198]:
cols = ["league", "split", "teamname", "opp_teamname", "ban1", "ban2", "ban3", "ban4", "ban5", "pick1", "pick2", "pick3", "pick4", "pick5"]

teams_train[cols] = teams_train[cols].astype("category")
teams_test[cols] = teams_test[cols].astype("category")

# 예측 모델 로드 및 예측

In [199]:
stats_columns = [
    "result",
    "gamelength",
    "kills",
    "deaths",
    "assists",
    "firstblood",
    "team kpm",
    "ckpm",
    "firstdragon",
    "firstherald",
    "void_grubs",
    "firstbaron",
    "firsttower",
    "towers",
    "firstmidtower",
    "firsttothreetowers",
    "turretplates",
    "inhibitors",
    "damagetochampions",
    "dpm",
    "damagetakenperminute",
    "damagemitigatedperminute",
    "wardsplaced",
    "wpm",
    "wardskilled",
    "wcpm",
    "controlwardsbought",
    "visionscore",
    "vspm",
]

In [200]:
train_data = pd.concat([teams_train, teams_test], ignore_index=True)
target_data = pd.concat([teams_train_target, teams_test_target], ignore_index=True)

In [201]:
import joblib

model1 = joblib.load("output/stacking_0107.pkl")
model2 = joblib.load("output/cat_0107.pkl")

## 입력 데이터 특성 추가

In [202]:
teamname = "Dplus KIA"
opp_teamname = "OKSavingsBank BRION"
patch = 14.23
league = "LCK"
side = "Blue"
year = 2024
month = 12
day = 8
ban1 = "Vi"
ban2 = "Renekton"
ban3 = "Skarner"
ban4 = "Ashe"
ban5 = "Kalista"
pick1 = "K'Sante"
pick2 = "Viego"
pick3 = "Aurora"
pick4 = "Varus"
pick5 = "Maokai"

input_data = {
    "teamname": teamname,
    "opp_teamname": opp_teamname,
    "patch": patch,
    "league": league, 
    "side": side,
    "year": year,
    "month": month,
    "day": day,
    "ban1": ban1,
    "ban2": ban2,
    "ban3": ban3,
    "ban4": ban4,
    "ban5": ban5,
    "pick1": pick1,
    "pick2": pick2,
    "pick3": pick3,
    "pick4": pick4,
    "pick5": pick5
}

In [203]:
input_team_data = train_data[train_data['teamname'] == teamname].sort_values(['year', 'month', 'day']).reset_index(drop=True)
input_opp_data = train_data[train_data['teamname'] == opp_teamname].sort_values(['year', 'month', 'day']).reset_index(drop=True)

recent10_stats = {}
for col in stats_columns:
    if len(input_team_data) > 0:
        team_recent10 = train_data[train_data['teamname'] == teamname][col].rolling(window=10, min_periods=1).mean().iloc[-1]
    else:
        team_recent10 = 0.5 if col == 'result' else 0
    recent10_stats[f'recent10_{col}'] = team_recent10
    
    if len(input_opp_data) > 0:
        opp_recent10 = train_data[train_data['teamname'] == opp_teamname][col].rolling(window=10, min_periods=1).mean().iloc[-1]
    else:
        opp_recent10 = 0.5 if col == 'result' else 0
    recent10_stats[f'opp_recent10_{col}'] = opp_recent10

for feature in recent10_stats:
    input_data[feature] = recent10_stats[feature]

In [204]:
head_to_head = train_data[
    ((train_data['teamname'] == teamname) & (train_data['opp_teamname'] == opp_teamname)) |
    ((train_data['teamname'] == opp_teamname) & (train_data['opp_teamname'] == teamname))
].sort_values(['year', 'month', 'day'])

if len(head_to_head) > 0:
    team_wins = head_to_head[
        ((head_to_head['teamname'] == teamname) & (head_to_head['result'] == 1) |
        (head_to_head['teamname'] == opp_teamname) & (head_to_head['result'] == 0))
    ].shape[0]
    h2h_winrate = team_wins / len(head_to_head)
else:
    h2h_winrate = 0.5

input_data['h2h_winrate'] = h2h_winrate

In [205]:
league_stats = {}

team_league_games = train_data[
    (train_data['teamname'] == teamname) & 
    (train_data['league'] == input_data['league'])
]
if len(team_league_games) > 0:
    team_league_winrate = team_league_games['result'].mean()
else:
    team_league_winrate = 0.5

opp_league_games = train_data[
    (train_data['teamname'] == opp_teamname) & 
    (train_data['league'] == input_data['league'])
]
if len(opp_league_games) > 0:
    opp_league_winrate = opp_league_games['result'].mean()
else:
    opp_league_winrate = 0.5

input_data['league_winrate'] = team_league_winrate
input_data['opp_league_winrate'] = opp_league_winrate

In [206]:
input_data = pd.DataFrame([input_data.values()], columns=input_data.keys())
input_data

,teamname,opp_teamname,patch,league,side,year,month,day,ban1,ban2,...,opp_recent10_wcpm,recent10_controlwardsbought,opp_recent10_controlwardsbought,recent10_visionscore,opp_recent10_visionscore,recent10_vspm,opp_recent10_vspm,h2h_winrate,league_winrate,opp_league_winrate
0,Dplus KIA,OKSavingsBank BRION,14.23,LCK,Blue,2024,12,8,Vi,Renekton,...,1.69032,57.7,45.2,319.4,281.2,9.00584,8.75083,0.8,0.573333,0.292308


In [169]:
input_data["side"] = input_data["side"].map({"Blue": 0, "Red": 1})
cat_input_data = input_data.copy()

In [170]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def preprocess(input_data, train_data):
    champion_columns_teams = ['ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'pick1', 'pick2', 'pick3', 'pick4', 'pick5']

    champions = sorted(train_data[champion_columns_teams].stack().unique())

    champions_df = pd.DataFrame({'champion': champions})
    champions_df = champions_df.dropna().reset_index(drop=True)

    le = LabelEncoder()
    champions_df['champion_encoded'] = le.fit_transform(champions_df['champion'])

    for col in champion_columns_teams:
        input_data[col] = le.transform(input_data[col])
        
    encoder = OneHotEncoder()
    league_encoded = encoder.fit_transform(input_data[["league"]]).toarray()
    league_cols = [f"league_{col}" for col in encoder.categories_[0]]
    input_data = pd.concat(
        [input_data, pd.DataFrame(league_encoded, columns=league_cols)], axis=1
    )
    input_data.drop("league", axis=1, inplace=True)

    le_team = LabelEncoder()
    all_team_names = pd.concat(
        [
            train_data["teamname"],
            train_data["opp_teamname"],
        ]
    )
    le_team.fit(all_team_names)

    input_data["teamname"] = le_team.transform(input_data["teamname"])
    input_data["opp_teamname"] = le_team.transform(input_data["opp_teamname"])
    
    return input_data

input_data = preprocess(input_data, train_data)

In [155]:
from sklearn.preprocessing import MinMaxScaler


def scale(input_data, train_data):
    scaler = MinMaxScaler()
    numeric_cols = input_data.select_dtypes("number").columns

    scaler.fit(train_data[numeric_cols])
    input_data[numeric_cols] = scaler.transform(input_data[numeric_cols])
    
    return input_data


input_data = scale(input_data, train_data)

KeyError: "['recent10_result', 'opp_recent10_result', 'recent10_gamelength', 'opp_recent10_gamelength', 'recent10_kills', 'opp_recent10_kills', 'recent10_deaths', 'opp_recent10_deaths', 'recent10_assists', 'opp_recent10_assists', 'recent10_firstblood', 'opp_recent10_firstblood', 'recent10_team kpm', 'opp_recent10_team kpm', 'recent10_ckpm', 'opp_recent10_ckpm', 'recent10_firstdragon', 'opp_recent10_firstdragon', 'recent10_firstherald', 'opp_recent10_firstherald', 'recent10_void_grubs', 'opp_recent10_void_grubs', 'recent10_firstbaron', 'opp_recent10_firstbaron', 'recent10_firsttower', 'opp_recent10_firsttower', 'recent10_towers', 'opp_recent10_towers', 'recent10_firstmidtower', 'opp_recent10_firstmidtower', 'recent10_firsttothreetowers', 'opp_recent10_firsttothreetowers', 'recent10_turretplates', 'opp_recent10_turretplates', 'recent10_inhibitors', 'opp_recent10_inhibitors', 'recent10_damagetochampions', 'opp_recent10_damagetochampions', 'recent10_dpm', 'opp_recent10_dpm', 'recent10_dama